<a href="https://colab.research.google.com/github/TheCaduceus/Multi-Cloud-Transfer-Tool/blob/main/Rclone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**First connect Google Drive, to enable MCT Tool to save/retrive its configuration files.**

In [ ]:
#@markdown <br><center><img src="https://logosmarcas.net/wp-content/uploads/2020/12/Google-Drive-Logo.png" height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Connect to Google Account</h3></center><br>

from pathlib import Path
from contextlib import suppress
from sys import exit as end
from requests import get as get_url
from google.colab import output
from google.colab.drive import mount, flush_and_unmount as unmount

current_version = "v5.0.2"

# Check for latest version
try:
	json_data = get_url("https://gist.githubusercontent.com/TheCaduceus/29789e6972aad136a99b35fe0d9adbde/raw/MCT.json").json()
	if current_version == json_data["version"]:
		print(f'Great Job! you are using latest version: {json_data["version"]}')
	else:
		print(f'Oh-No! you are not using latest version and missing out important fixes & features. Latest version is {json_data["version"]} & update it from:\n{json_data["release_link"]}')
except Exception as error_code:
	end(f'Unable to check for latest version!\nTraceback:\n{error_code}')

MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
FORCE_REMOUNT = True #@param {type:"boolean"}
CUSTOM_MOUNT_POINT = False #@param {type:"boolean"}
DEBUG_MODE = False #@param {type:"boolean"}

if CUSTOM_MOUNT_POINT:
  Custom_Mount_Path = input('Please enter Custom Mount Path:\n')
  if not Custom_Mount_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Mount_Path).exists():
    end("Path Error: Provided Custom path not found! Check if you mounted/connected it or if you having reading/writing permission?")
  elif not Path(Custom_Mount_Path).is_dir():
    end("Path Error: Given Path is not a valid directory.")
else:
  Custom_Mount_Path = '/content/drive'

mount._DEBUG = DEBUG_MODE

if MODE == "MOUNT":
  print("Please Wait! Mounting...")
  mount(Custom_Mount_Path, force_remount= FORCE_REMOUNT)
  if not DEBUG_MODE:
    output.clear()
  print(f'Mounted at {Custom_Mount_Path}')
elif MODE == 'UNMOUNT':
  with suppress(ValueError):
    print('Please Wait! UNMOUNT in progress...')
    unmount()
  get_ipython().system_raw('rm -rf /root/.config/Google/DriveFS')

<b>Options:</b><br>
`FORCE_REMOUNT` :<b> Forcefully remount Google Drive.</b><br>
`CUSTOM_MOUNT_POINT` :<b> Choose different path for mounting Google Drive.</b><br>
`DEBUG_MODE` :<b> To enable or disable DEBUG Mode.</b>

**Download setup files & configure the environment:**

<b>Options:</b><br>
`Force_Setup`<b>: Forcefully install Rclone by excluding all other dependencies and download them separately. Useful if any dependency (except Rclone) having temporary setup problem.</b><br>
`Custom_sh_file`:<b> Get custom .sh file from external source (URL or File Path) and execute it to install additional dependecies or requirements with or without normal setup.</b>

In [ ]:
#@markdown <br><center><img src="https://thecaduceus.github.io/Img/Setup_img.png" height="50" alt="Setup-Logo"/></center>
#@markdown <center><h3><b>Setup & Configure Environment</b></h3></center><br>

from sys import exit as end
from pathlib import Path, PurePath
from time import sleep
from requests import get as get_url
from urllib.parse import urlparse as validate_url
from importlib.util import find_spec as find_pkg
from google.colab.output import clear as clear_screen

current_version = "v5.0.2"
do_normal_setup = "1"

# Check for latest version
try:
	json_data = get_url("https://gist.githubusercontent.com/TheCaduceus/29789e6972aad136a99b35fe0d9adbde/raw/MCT.json").json()
	if current_version == json_data["version"]:
		print(f'Great Job! you are using latest version: {json_data["version"]}')
	else:
		print(f'Oh-No! you are not using latest version and missing out important fixes & features. Latest version is {json_data["version"]} & update it from:\n{json_data["release_link"]}')
except Exception as error_code:
	end(f'Unable to check for latest version!\nTraceback:\n{error_code}')

# Flavor Flags  
Stable = ""
Beta = "-s beta"

# User Input
Install_Rclone_Flavor = Stable #@param ["Stable", "Beta"] {type:"raw"}
Force_Setup = False #@param {type:"boolean"}
Custom_sh_file = False #@param {type:"boolean"}

if Custom_sh_file:

	Custom_sh_Source = input("Provide the Custom Install.sh file source, it can be URL or File Path:\n")

	use_sudo = input("Do you want to execute the .sh file as sudo? 1 = Yes and 0 = No:\n")

	if use_sudo not in ['0','1']:
		end("Unable to check for latest version!")
	elif not Custom_sh_Source:
		end("Custom sh file source can't be blank!")
	elif all([validate_url(Custom_sh_Source).scheme, validate_url(Custom_sh_Source).netloc]):
		try:
			get_url(Custom_sh_Source)
			if use_sudo == '1':
				!curl {Custom_sh_Source} | sudo bash
				print('Execution completed as sudo!')
			elif use_sudo == '0':
				!curl {Custom_sh_Source} | bash
				print('Execution completed!')
		except ConnectionError:
			end("Unable to connect with given sh file URL.")
		except Exception as error_code:
			end(f"Failed to complete the execution of .sh file!\nTraceback:\n{error_code}")
	elif Path(Custom_sh_Source).exists() and Path(Custom_sh_Source).is_file() and PurePath(Custom_sh_Source).suffix == '.sh':
		try:
			if use_sudo == '1':
				!sudo bash {Custom_sh_Source}
				print('Execution completed as sudo!')
			elif use_sudo == '0':
				!bash {Custom_sh_Source}
				print('Execution completed!')
		except Exception as error_code:
			end(f"Failed to complete the execution of .sh file!\nTraceback:\n{error_code}")
	elif not Path(Custom_sh_Source).exists():
		end("Provided Custom sh file path not exist!")
	elif Path(Custom_sh_Source).exists() and not Path(Custom_sh_Source).is_file():
		end("Provided Custom sh file path should of a particular file not directory/folder.")
	elif Path(Custom_sh_Source).exists() and Path(Custom_sh_Source).is_file() and not PurePath(Custom_sh_Source).suffix == '.sh':
		end("File format should be .sh")
	else:
		print('Unhandled error occured with .sh file!')

	# Ask for Normal Setup
	do_normal_setup = input('Do you want to run Normal Setup also? 1 = Yes and 0 = No:\n')

	if do_normal_setup not in ['0','1']:
		end("Input Error: Value should be 1 or 0")

# Force Setup
if Force_Setup:
	if Path("/content/setup.sh").exists():
		!rm {"/content/setup.sh"}
	print("Starting Force Setup...")
	sleep(4)
	!curl "https://rclone.org/install.sh" | sudo bash {Install_Rclone_Flavor}

if do_normal_setup == '1' and not Path("/content/setup.sh").exists():
	try:
		get_url(json_data["setup_file"])
		print("Connected to the Source.")
		!curl {json_data["setup_file"]} | sudo bash {Install_Rclone_Flavor}
		!curl {json_data["setup_file"]} > "/content/setup.sh"
		clear_screen()
		print("Setup completed!")
	except ConnectionError:
		print('Unable to get setup file!')
	except Exception as error_code:
		print(f'Setup instructions seems to be having mistake(s)!\nTraceback:\n{error_code}')
elif do_normal_setup == '1' and Path("/content/setup.sh").exists():
	print("Setup file exist and configured already!")

<b>Options:</b><br>
<b>By default, MCT Tool uses MCT-Config folder located in My Drive to restore configurations.</b><br>
`Custom_Import_Path` :<b> Scan a custom path (or directory) give by user for usable MCT Configuration files.</b><br>
`Custom_Export_Path` :<b> Export all your MCT configuration files to a gives path.</b>

In [ ]:
#@markdown <br><center><img src="https://1.bp.blogspot.com/-M5PLcSana6M/XgBHF7jUjiI/AAAAAAAAUzs/S24qhuijluwKlzIOnc2gntoI-U83ZsrJACLcBGAsYHQ/s1600/rclone_logo.png" height="50" alt="Rclone-Logo' "/></center>
#@markdown <center><h3><b>Fetch Previous MCT Config from My Drive</b></h3></center><br>

from sys import exit as end
from requests import get as get_url
from pathlib import Path, PurePath
from google.colab.output import clear as clear_screen

current_version = "v5.0.2"

# Check for latest version
try:
	json_data = get_url("https://gist.githubusercontent.com/TheCaduceus/29789e6972aad136a99b35fe0d9adbde/raw/MCT.json").json()
	if current_version == json_data["version"]:
		print(f'Great Job! you are using latest version: {json_data["version"]}')
	else:
		print(f'Oh-No! you are not using latest version and missing out important fixes & features. Latest version is {json_data["version"]} & update it from:\n{json_data["release_link"]}')
except Exception as error_code:
	end(f'Unable to check for latest version!\nTraceback:\n{error_code}')

# Where user mounted Google Drive?
try:
  CMP = PurePath(Custom_Mount_Path)
except NameError:
  CMP = '/content/drive'

DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config/"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
  end('Please download Setup first!')
elif not Path(CMP).exists():
  end('Not Found: Google Drive is not connected!')

#@markdown ---
Custom_Import_Path = False #@param {type:"boolean"}
Custom_Export_Path = False #@param {type:"boolean"}

# Import Configuration(s)

if Custom_Import_Path:
  Custom_Path = input('Enter the directory path from where configuration file(s) needs to be imported:\n')
  # Check Path
  if not Custom_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Path).exists():
    end("Path Error: Provided Custom path not found! Check if you mounted/connected it or if you having reading/writing permission?")
  elif not Path(Custom_Path).is_dir():
    end("Path Error: Given Path is not a valid directory.")
  elif not Path(f'{CMP}/MyDrive/MCT-Config/').exists():
    !rclone mkdir "$DRIVE_CONFIG_DIR"
    print('Created: MCT-Config [folder]')

  Custom_Path = PurePath(Custom_Path) # Remove any extra '/'

  # Paths
  IMPORT_TO = DRIVE_CONFIG_DIR
  IMPORT_RCLONE_CONF = f"{Custom_Path}/rclone.conf"
  IMPORT_ACCOUNTS_ZIP = f"{Custom_Path}/accounts.zip"
  IMPORT_ACCOUNTS_FOLDER = f"{Custom_Path}/accounts/"
  ZIP_ACCOUNTS_FOLDER = f"{CMP}/MyDrive/MCT-Config/accounts" # accounts is name of zip, zip file will be saved in MCT-Config
  IMPORT_MEGA_JSON = f"{Custom_Path}/mega.json"
  IMPORT_GOFILE_JSON = f"{Custom_Path}/gofile.json"

  # Check 'rclone.conf' file

  if Path(IMPORT_RCLONE_CONF).exists():

    is_import = input('Found: rclone.conf [file] | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_RCLONE_CONF" "$IMPORT_TO"
      print('Successfully imported "rclone.conf" file! (1)')
    else:
      print('Not Found: rclone.conf [file]')

  # Check 'Service Accounts' file/folder

  if Path(IMPORT_ACCOUNTS_ZIP).exists() and Path(IMPORT_ACCOUNTS_FOLDER).exists():

    choice = input('Found both! "accounts.zip" [file] & "accounts" [folder]\n Select one: 0 = accounts.zip [file] or 1 = accounts [folder] or 2 = Ignore\n')

    if choice not in ['0','1','2']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif choice == '1':
      !zip "$ZIP_ACCOUNTS_FOLDER" "$IMPORT_ACCOUNTS_FOLDER"
      print('Successfully imported "accounts.zip" file! (2)')
    elif choice == '0':
      !rclone copy "$IMPORT_ACCOUNTS_ZIP" "$IMPORT_TO"
      print('Successfully imported "accounts.zip" file! (1)')
  elif Path(IMPORT_ACCOUNTS_ZIP).exists():

    is_import = input('Found: accounts.zip [file] | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_ACCOUNTS_ZIP" "$IMPORT_TO"
      print('Successfully imported "accounts.zip" file! (1)')
  elif Path(IMPORT_ACCOUNTS_FOLDER).exists():

    is_import = input('Found: accounts [folder] | Do you want to import ? 0=No & 1=Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !zip "$ZIP_ACCOUNTS_FOLDER" "$IMPORT_ACCOUNTS_FOLDER"
      print('Successfully imported "accounts.zip" file! (2)')
  else:
    print('Not Found: accounts.zip [file] or accounts [folder]')

  # Check 'mega.json' file

  if Path(IMPORT_MEGA_JSON).exists():

    is_import = input('Found: mega.json | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_MEGA_JSON" "$IMPORT_TO"
      print('Successfully imported "mega.json" file! (1)')
        
  else:
    print('Not Found: mega.json [file]')

  # Check 'gofile.json' file

  if Path(IMPORT_GOFILE_JSON).exists():

    is_import = input('Found: gofile.json | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_GOFILE_JSON" "$IMPORT_TO"
      print('Successfully imported "gofile.json" file! (1)')
        
  else:
    print('Not Found: gofile.json [file]')

# Restore Configuration(s) from MCT-Config folder

if not Path(DRIVE_CONFIG_DIR).exists():
  end('Not Found: MCT-Config [folder]')
else:
  print('Found: MCT-Config [folder]')

# Paths
ROOT = "/content/"
DRIVE_RCLONE_CONF = f"{CMP}/MyDrive/MCT-Config/rclone.conf"
ROOT_RCLONE_CONF = "/root/.config/rclone/"
DRIVE_ACCOUNTS_ZIP = f"{CMP}/MyDrive/MCT-Config/accounts.zip"
DRIVE_MEGA_JSON = f"{CMP}/MyDrive/MCT-Config/mega.json"
DRIVE_GOFILE_JSON = f"{CMP}/MyDrive/MCT-Config/gofile.json"

# Check 'rclone.conf' file

if Path(DRIVE_RCLONE_CONF).exists():
  !rclone copy "$DRIVE_RCLONE_CONF" "$ROOT_RCLONE_CONF"
  print('Restored: rclone.conf [file]')
else:
  print('Not Found: rclone.conf [file]')

# Check 'Service Accounts' file

if Path(DRIVE_ACCOUNTS_ZIP).exists():
  !rclone copy "$DRIVE_ACCOUNTS_ZIP" "$ROOT"
  print('Restored: accounts.zip [file]')
else:
  print('Not Found: accounts.zip [file]')

# Check 'mega.json' file

if Path(DRIVE_MEGA_JSON).exists():
  !rclone copy "$DRIVE_MEGA_JSON" "$ROOT"
  print('Restored: mega.json [file]')
else:
  print('Not Found: mega.json [file]')

# Check 'gofile.json' file

if Path(DRIVE_GOFILE_JSON).exists():
  !rclone copy "$DRIVE_GOFILE_JSON" "$ROOT"
  print('Restored: gofile.json [file]')
else:
  print('Not Found: gofile.json [file]')

if Custom_Export_Path:

  #Paths
  EXPORT_TO = f"{Custom_Path}/MCT-Config/"

  Custom_Path = input('Enter the directory path to where configuration file(s) needs to be exported:\n')

  if not Custom_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Path).exists():
    end("Path Error: Path not found!")
  elif not Path(Custom_Path).is_dir():
    end("Path Error: Path is not a valid directory.")
  elif not Path(DRIVE_CONFIG_DIR):
    !rclone mkdir "$DRIVE_CONFIG_DIR"

  # Check 'rclone.conf' file

  if Path(DRIVE_RCLONE_CONF).exists():

    is_export = input('Found: rclone.conf [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_RCLONE_CONF" "$EXPORT_TO"
      print('Export: rclone.conf [file] (1)')
  else:
    print('Not Found: rclone.conf [file]')

    # Check 'Service Accounts' file

  if Path(DRIVE_ACCOUNTS_ZIP).exists():

    is_export = input('Found accounts.zip [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_ACCOUNTS_ZIP" "$EXPORT_TO"
      print('Export: accounts.zip [file] (1)')
  else:
    print('Not Found: accounts.zip [file]')

  # Check 'mega.json' file

  if Path(DRIVE_MEGA_JSON).exists():

    is_export = input('Found mega.json [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_MEGA_JSON" "$EXPORT_TO"
      print('Export: mega.json [file] (1)')
  else:
    print('Not Found: mega.json [file]')

  # Check 'gofile.json' file

  if Path(DRIVE_GOFILE_JSON).exists():

    is_export = input('Found gofile.json [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0', '1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_GOFILE_JSON" "$EXPORT_TO"
      print('Export: gofile.json [file] (1)')
  else:
    print('Not Found: gofile.json [file]')

**Using Service Accounts:<br> Just zip the "accounts" folder which have 100 or more json files and upload it using the below code. Name of zip should be ``accounts.zip``. Then copy the path of json file and use it to create Rclone Config file below. (Optional)** ***(Refer Guide for detailed help)***<br>
****
**Saving Options:**<br>
``Upload_Service_Accounts`` **: Upload new Service Accounts.**<br>
``Save_To_MyDrive`` **: Save new/existing Service Accounts to Google Drive-> My Drive**

In [ ]:
# @markdown <br><center><img src="https://pluspng.com/img-png/google-cloud-logo-png-google-cloud-logo-evolution-history-and-meaning-png-1280x720.png" height="150" alt="Rclone-Logo' "/></center>
# @markdown <center><h3><b>Using Service Accounts</b></h3></center><br>

from sys import exit as end
from requests import get as get_url
from pathlib import Path, PurePath
from google.colab import files

current_version = "v5.0.2"

# Check for latest version
try:
	json_data = get_url("https://gist.githubusercontent.com/TheCaduceus/29789e6972aad136a99b35fe0d9adbde/raw/MCT.json").json()
	if current_version == json_data["version"]:
		print(f'Great Job! you are using latest version: {json_data["version"]}')
	else:
		print(f'Oh-No! you are not using latest version and missing out important fixes & features. Latest version is {json_data["version"]} & update it from:\n{json_data["release_link"]}')
except Exception as error_code:
	end(f'Unable to check for latest version!\nTraceback:\n{error_code}')

# Where user mounted Google Drive?
try:
	CMP = PurePath(Custom_Mount_Path)
except NameError:
	CMP = '/content/drive'

# Paths
ROOT = "/content/"
DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config/"
ROOT_ACCOUNTS_ZIP = "/content/accounts.zip"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
	end('Please download Setup first!')
elif not Path(CMP).exists():
	end('Not Found: Google Drive is not connected!')
elif not Path(DRIVE_CONFIG_DIR).exists():
	!rclone mkdir {DRIVE_CONFIG_DIR}
	print('Created: MCT-Config [folder] -> MyDrive')

# Configure Options
Upload_Service_Accounts = True  # @param {type:"boolean"}
Save_To_MyDrive = True  # @param {type:"boolean"}

if Upload_Service_Accounts and Save_To_MyDrive:
	print('Upload accounts.zip:\n')
	file = files.upload()
	if list(file.keys())[0] == 'accounts.zip':
		!rclone copy {ROOT_ACCOUNTS_ZIP} {DRIVE_CONFIG_DIR}
		!unzip {ROOT_ACCOUNTS_ZIP} -d {ROOT}
		print('Uploaded & Configured Service Accounts! (2)')
	else:
		end('File Error: File is not accounts.zip')

elif not Upload_Service_Accounts and not Save_To_MyDrive:
	print('Task Error: No Task to perform :)')

elif Upload_Service_Accounts:
	print('Upload accounts.zip:\n')
	file = files.upload()
	if list(file.keys())[0] == 'accounts.zip':
		!unzip {ROOT_ACCOUNTS_ZIP} -d {ROOT}
		print('Uploaded & Configured Service Accounts! (1)')
	else:
		end('File Error: File is not accounts.zip')

elif Path(ROOT_ACCOUNTS_ZIP).exists():
	print('Found: accounts.zip [file]')
	!unzip {ROOT_ACCOUNTS_ZIP} -d {ROOT}
	print('Configured Service Accounts!')

else:
	print('Not Found: accounts.zip [file]')

**Create/Edit/View/Delete your Rclone Config file:<br>``Upload_Config_File`` : Upload your already made Rclone config file to use it.<br>``Save_To_MyDrive`` : To save created Rclone config file in Google Drive's My Drive.<br>``Download_Config`` : Download created Rclone config file in your Device.**


In [ ]:
#@markdown <br><center><img src="https://1.bp.blogspot.com/-M5PLcSana6M/XgBHF7jUjiI/AAAAAAAAUzs/S24qhuijluwKlzIOnc2gntoI-U83ZsrJACLcBGAsYHQ/s1600/rclone_logo.png" height="50" alt="Rclone-Logo' "/></center>
#@markdown <center><h3><b>Create/Edit/Delete Rclone Config File</b></h3></center><br>

from google.colab import files, output
from sys import exit as end
from time import sleep
from pathlib import Path, PurePath

# Where user mounted Google Drive?
try:
  CMP = PurePath(Custom_Mount_Path)
except NameError:
  CMP = '/content/drive'

# Paths
DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
  end('Please download Setup first!')
elif not Path(f'{CMP}').exists():
  end('Not Found: Google Drive is not connected!')
elif not Path(DRIVE_CONFIG_DIR).exists():
  !rclone mkdir {DRIVE_CONFIG_DIR}
  print('Created: MCT-Config [folder] -> MyDrive')

#@markdown ****
Upload_Config_File = False #@param {type:"boolean"} 
Save_To_MyDrive = True #@param {type:"boolean"}
Download_Config = False #@param {type:"boolean"}

if Upload_Config_File and Save_To_MyDrive and Download_Config:
  # Initiate Upload Request
  print('Request: Initiate upload -> rclone.conf [file]')
  file = files.upload()
  if list(file.keys())[0] != 'rclone.conf':
    end('File Error: File is not rclone.conf')
  elif not Path("/root/.config/rclone/").exists():
    !rclone mkdir "/root/.config/rclone/"
  !rclone copy "/content/rclone.conf" "/root/.config/rclone/"
  print('Saved: rclone.conf [file] -> Runtime')
  # Rclone configuration
  !rclone config
  # Save to Google Drive
  !rclone copy "/content/rclone.conf" {DRIVE_CONFIG_DIR}
  print('Saved: rclone.conf [file] -> MyDrive')
  # Initiate Download Request
  files.download("/root/.config/rclone/")
  print('Request: Initiated download -> rclone.conf [file]')
elif Upload_Config_File and Save_To_MyDrive:
  # Initiate Upload Request
  print('Request: Initiate upload -> rclone.conf [file]')
  file = files.upload()
  if list(file.keys())[0] != 'rclone.conf':
    end('File Error: File is not rclone.conf')
  elif not Path('/root/.config/rclone/').exists():
    !rclone mkdir "/root/.config/rclone/"
  !rclone copy "/content/rclone.conf" "/root/.config/rclone/"
  print('Saved: rclone.conf [file] -> Runtime')
  # Rclone configuration
  !rclone config 
  # Save to Google Drive
  !rclone copy "/content/rclone.conf" {DRIVE_CONFIG_DIR}
  print('Saved: rclone.conf [file] -> MyDrive')
elif Upload_Config_File:
  # Initiate Upload Request
  print('Request: Initiate Upload -> rclone.conf [file]')
  file = files.upload()
  if list(file.keys())[0] != 'rclone.conf':
    end('File Error: File is not rclone.conf')
  elif not Path('/root/.config/rclone/').exists():
    !rclone mkdir "/root/.config/rclone/"
  print('[WARNING] -> No saving option selected, progress may lose.')
  # Rclone configuration
  !rclone config
elif Save_To_MyDrive:
  # Rclone configuration
  !rclone config 
  # Save to Google Drive
  !rclone copy "/content/rclone.conf" {DRIVE_CONFIG_DIR}
  print('Saved: rclone.conf [file] -> MyDrive')
elif Download_Config:
  # Rclone configuration
  !rclone config 
  # Initiate Download Request
  files.download("/root/.config/rclone/")
  print('Request: Initiated download -> rclone.conf [file]')
else:
  print('[WARNING] -> No saving option selected, progress may lose.')
  !rclone config

**Transfer Tools:**

**Connect your cloud accounts, between which you want to transfer your data, run the following code. It will show the names of configured cloud accounts in your Rclone config file to mount them.**

**Options:**<br>
`Custom_Mount_Point`**: To set Custom Mount Point.**<br>
`Custom_Cache_Dir`**: To set Custom Cache Directory.**

In [ ]:
# @markdown <br>
# @markdown <center><h3><b>Connect To Cloud Accounts</b></h3></center>
# @markdown <br><center><img src="https://cdn1.iconfinder.com/data/icons/cloud-system-color-line/64/cluster-512.png" height="100" alt="Rclone-Logo' "/></center>
# @markdown <center><b>Connect your Cloud Accounts (Through Rclone Config file!) between which you want to transfer Data!</b></center>

from sys import exit as end
from os import makedirs, system
from pathlib import Path, PurePath
from re import findall, M

if not Path("/root/.config/rclone/rclone.conf").exists():
  end('Not Found: rclone.conf [file]')

content = open("/root/.config/rclone/rclone.conf").read()
avCon = findall(r"^\[(.+)\]$", content, M)

print("Available Rclone remotes:")
print(*avCon)
remote = input("Enter Rclone remote name to continue:\n")

if remote not in avCon:
  end('Not Found: Given remote name not exist in rclone.conf [file]')

# User Input
Custom_Mount_Point = False  # @param {type:"boolean"}
Custom_Cache_Dir = False  # @param {type:"boolean"}

# Custom Mount Point
if Custom_Mount_Point:
  Mount_Point = input("Enter Custom Mount path:\n")
  if not Mount_Point:
    end("Mount Path can't be empty.")
  elif Path(Mount_Point).exists() and Path(Mount_Point).is_dir():
      Mount_Point = f"{PurePath(Mount_Point)}/{remote}"
  elif not Path(Mount_Point).exists():
      end('Not Found: Given mount path not found.')
  elif not Path(Mount_Point).is_dir():
    end('Path Error: Given path is not a valid directory.')
else:
  Mount_Point = f"/content/drives/{remote}"

# Custom Cache Directory
if Custom_Cache_Dir:
  Cache_dir = input("Enter Custom Cache Directory path:\n")
  if not Cache_dir:
    end("Cache Directory Path can't be empty.")
  elif Path(Cache_dir).exists() and Path(Cache_dir).is_dir():
    Cache_dir = f"{PurePath(Cache_dir)}/{remote}"
  elif not Path(Cache_dir).exists():
    end('Not Found: Given mount paht not found.')
  elif not Path(Cache_dir).exists():
    end('Path Error: Give path is not a valid directory.')
else:
  Cache_dir = "/content/temp/Rclone"

Action = input("Select Action: 1 = Mount and 0 = Unmount:\n")

if Action not in ['0','1']:
  end("Input Error: Value should be 1 or 0.")
elif Action == '1':
  makedirs(Mount_Point, exist_ok=True)
  !rclone mount {remote}: {Mount_Point} --user-agent 'Mozilla' --buffer-size 256M --transfers 10 --vfs-cache-mode minimal --vfs-read-chunk-size 500M --vfs-cache-max-size 50G --vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s --cache-dir {Cache_dir} --allow-other --daemon
  print(f'Mounted: {Mount_Point}') if Path(Mount_Point).exists() else print(f"Mount Error: Unable to mount {Mount_Point}")
elif Action == '0' and system(f"fusermount -uz {Mount_Point}") == 0:
  !rm -r {Mount_Point}
else:
  system(f"fusermount -uz {Mount_Point}")

**Now Enter the Source & Destination Address and Notification settings to get notify once your Task got Completed! To get both Address click on File Icon.**

In [ ]:
# @markdown <br>
# @markdown <center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>

# @markdown ---
import smtplib
source = ""  # @param {type:"string"}
destination = ""  # @param {type:"string"}
mode = "copy"  # @param ["copy", "copyto", "copyurl", "move", "moveto", "check", "sync", "bisync", "cat", "mkdir","cleanup", "ls", "lsd", "lsl", "test makefile","md5sum", "sha1sum", "size", "genautocomplete", "gendocs", "tree","obscure"]
dry_run = False  # @param {type:"boolean"}
Extra_args = ""  # @param {type:"string"}
# @markdown ---
Email_notification = False  # @param {type:"boolean"}
logs = False  # @param {type:"boolean"}
emailID = ""  # @param {type:"string"}
password = ""  # @param {type:"string"}
Receiver_ID = ""  # @param {type:"string"}
# @markdown ---

args = "--transfers 20 --checkers 20 --stats-one-line --stats=5s -v --tpslimit 95 --tpslimit-burst 40 "
if mode == "check":
  args += "--one-way "
if dry_run:
    args += " --dry-run "
if logs:
  args += " --log-file Rclone_log.txt "
if Extra_args != "":
  args += Extra_args

def runRclone():
  !rclone --user-agent "Mozilla" "$mode" "$source" "$destination" $args


def checkEmail():
  if Email_notification and not logs:
    print("You will receive only a notification after task has finished.")
    runRclone()
    s = _extracted_from_checkEmail_7(emailID)
    message = "Your rClone task has Completed!"
    s.sendmail(emailID, Receiver_ID, message)
    print("Email Alert Sent!")
    s.quit()
  elif Email_notification and logs == True:
    print("You will receive a notification with log attached after task has finished.")
    runRclone()
    # Sending email notification with logs

    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.base import MIMEBase
    from email import encoders

    fromaddr = emailID
    toaddr = Receiver_ID
    msg = MIMEMultipart()
    # storing the senders email address
    msg['From'] = fromaddr
    # storing the receivers email address
    msg['To'] = toaddr
    # storing the subject
    msg['Subject'] = "Colab has Finished Running your Cell"
    # string to store the body of the mail
    body = "Your Rclone Task has Completed!"

    msg.attach(MIMEText(body, 'plain'))
    filename = "Rclone_log.txt"
    attachment = open(f"/content/{filename}", "rb")
    p = MIMEBase('application', 'octet-stream')
    p.set_payload((attachment).read())
    # encode into base64
    encoders.encode_base64(p)
# p.add_header('Content-Disposition', "attachment; filename= %s" % filename)
    p.add_header('Content-Disposition', f"attachment; filename= {filename}")
    msg.attach(p)
    s = _extracted_from_checkEmail_7(fromaddr)
    text = msg.as_string()
    s.sendmail(fromaddr, toaddr, text)
    print("Email Alert Sent with log!")
    s.quit()
  elif (Email_notification == False and logs == False):
    print("You will not receive any notification!!!.")
    runRclone()

def _extracted_from_checkEmail_7(arg0):
  result = smtplib.SMTP('smtp.gmail.com', 587)
  result.starttls()
  result.login(arg0, password)
  return result

checkEmail()

**Delete your Files/Folder/Directory from below:<br>``delete`` Delete the content of folder or single file.<br>``purge`` Delete whole Directory and leaving nothing behind.<br>``rmdir`` Completely delete the given path.<br>``rmdirs`` Remove any empty directories under the path.**

In [ ]:
from time import sleep
from sys import exit as end
from pathlib import Path

# User Input
Action = "delete"  # @param ["delete", "purge", "rmdir", "rmdirs"]
Path = ""  # @param {type:"string"}
Arguments = ""  # @param {type:"string"}

if not Path:
    end("Path Error: Path can't be empty.")
elif Path(Path).exists():
    print(f'Action Selected: {Action}\n[INFO] - 5 Seconds left before executing action.')
    sleep(5)
    print('Executing...')
    !rclone {Action} {Path} {Arguments}
    print('Execution Completed!')
elif not Path(Path).exists():
    end("Path Error: Given path not exist.")
else:
    print('[ERROR] - Unhandled Action.')

###**Special Add-ons:**

In [ ]:
from libtorrent import session as lt_session, add_magnet_uri, torrent_info
from sys import exit as end
from requests import get as get_url, Session as req_session, post as post_url
from pathlib import Path, PurePath
from google.colab.files import download as dl
from os import path, makedirs, strerror

current_version = "v5.0.2"

# Check for latest version
try:
	json_data = get_url("https://gist.githubusercontent.com/TheCaduceus/29789e6972aad136a99b35fe0d9adbde/raw/MCT.json").json()
	if current_version == json_data["version"]:
		print(f'Great Job! you are using latest version: {json_data["version"]}')
	else:
		print(f'Oh-No! you are not using latest version and missing out important fixes & features. Latest version is {json_data["version"]} & update it from:\n{json_data["release_link"]}')
except Exception as error_code:
	end(f'Unable to check for latest version!\nTraceback:\n{error_code}')

# Where user mounted Google Drive?
try:
  CMP = PurePath(Custom_Mount_Path)
except NameError:
  CMP = '/content/drive'

# Paths
DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config/"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
  end('Please download Setup first!')
elif not Path(CMP).exists():
  end('Not Found: Google Drive is not connected!')
elif not Path(f'{CMP}/MyDrive/MCT-Config/').exists():
	!rclone mkdir "$DRIVE_CONFIG_DIR"
	print('Created: MCT-Config [folder] -> MyDrive')

# LibTorrent Initialization
ses = lt_session()
ses.listen_on(6881, 6891)
downloads = []

# User Input
Source = ""  # @param {type:"string"}
Mode = "YouTube Videos/Playlist & Other Sites" # @param ["Direct Download Links", "Download Links from txt file", "YouTube Videos/Playlist & Other Sites", "Magnet Links DL", "Torrent Files DL", "Upload file to Anonymous Platforms","MEGA Link DL", "UUP DUMP"]
Destination = ""  # @param {type:"string"}
args = ""  # @param {type:"string"}

def mega_dl():
  from subprocess import Popen, PIPE, STDOUT
  from contextlib import closing
  global Destination

  if not Destination:
    makedirs("downloads", exist_ok=True)
    Destination = "downloads"
  newlines = ['\n', '\r\n', '\r']
  def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with closing(stream):
      while True:
        out = []
        last = stream.read(1)
        if last == '' and proc.poll() is not None:
          break
        while last not in newlines:
          if last == '' and proc.poll() is not None:
            break
          out.append(last)
          last = stream.read(1)
        out = ''.join(out)
        yield out

  def transfer():
    cmd = ["mega-get", Source, f"{Destination}"]
    proc = Popen(
      cmd,
      stdout=PIPE,
      stderr=STDOUT,
      universal_newlines=True,
    )
    for line in unbuffered(proc):
      print(line)
  transfer()

def config_creds():
  from functools import wraps
  from errno import ETIME
  from signal import signal, alarm, SIGALRM
  from json import load as json_load
  from ocr import runSh
  class TimeoutError(Exception):
    pass

  def timeout(seconds=10, error_message=strerror(ETIME)):
    def decorator(func):
      def _handle_timeout(signum, frame):
        raise TimeoutError(error_message)

      def wrapper(*args, **kwargs):
        signal(SIGALRM, _handle_timeout)
        alarm(seconds)
        try:
          result = func(*args, **kwargs)
        finally:
          alarm(0)
        return result
      return wraps(func)(wrapper)
    return decorator

  @timeout(10)
  def runShT(args):
    return runSh(args, output=True)
  if not Path("/usr/bin/mega-cmd").exists():
    print("[ERROR] - MEGA not installed.")
  else:
    !pkill mega-cmd

  with open('mega.json', 'r') as mega_json:
    mega_json = json_load(mega_json)
  try:
    decrypted_mega_id = post_url(f'{json_data["apis"]["encrypt_api"]}/decrypt', json={"string": f"{mega_json['mega_id']}"}).json()
    decrypted_mega_pass = post_url(f'{json_data["apis"]["encrypt_api"]}/decrypt', json={"string": f"{mega_json['mega_pass']}"}).json()
  except Exception as error_code:
    end(f"Unable to decrypt MEGA Credentials.\nTraceback:\n{error_code}")


  try:
    runShT(f"mega-login {decrypted_mega_id['result']} {decrypted_mega_pass['result']}")
  except TimeoutError:
    runSh('mega-whoami', output=True)
  mega_dl()

def save_creds():
  from getpass import getpass
  from json import dumps as dump_json
  mega_id = input('Enter MEGA Email ID or Username:\n')
  mega_pass = getpass('Enter MEGA Account Password: (Input is hidden)\n')
  if not mega_id:
    end("Input Error: MEGA ID can't be blank!")
  elif not mega_pass:
    end("Input Error: MEGA Password can't be blank!")
  elif len(mega_pass) < 8:
    end("Input Error: MEGA Password must be of 8 digit or greater.")

  print('[INFO] - Encrypting MEGA Credentials')
  try:
    encrypted_mega_id = post_url(f'{json_data["apis"]["encrypt_api"]}/encrypt', json={"string": f"{mega_id}"}).json()
    encrypted_mega_pass = post_url(f'{json_data["apis"]["encrypt_api"]}/encrypt', json={"string": f"{mega_pass}"}).json()
  except Exception as error_code:
    end(f"Unable to encrypt MEGA Credentials.\nTraceback:\n{error_code}")
  encrypted_mega_json = {"mega_id": f"{encrypted_mega_id['result']}", "mega_pass": f"{encrypted_mega_pass['result']}"}
  with open('mega.json','w') as save_json:
    save_json.write(dump_json(encrypted_mega_json))
    print('Created: mega.json [file]')
    save_json.close()
  !rclone copy "/content/mega.json" "$DRIVE_CONFIG_DIR"
  print('Saved: mega.json -> MyDrive')
  config_creds()

if Mode =="Direct Download Links":
  links = Source.split(',')
  for link in links:
    %cd "$Destination"
    !aria2c "$link" $args
  print('Execution completed!')
elif Mode == "Download Links from txt file":
  %cd "$Destination"
  !aria2c -i "$Source" $args
  print('Execution completed!')
elif Mode == "YouTube Videos/Playlist & Other Sites":
  %cd "$Destination"
  !yt-dlp "$Source" $args
  print('Execution completed!')
elif Mode == "Magnet Links DL":
  params = {"save_path": Destination}
  while True:
    magnet_link = input("Enter Magnet Link Or Type Exit:\n")
    if magnet_link.lower() == "exit":
      break
    downloads.append(
      add_magnet_uri(ses, magnet_link, params)
    )

  from time import sleep
  from IPython.display import display
  from ipywidgets import Layout, FloatSlider
  state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
  ]

  layout = Layout(width="auto")
  style = {"description_width": "initial"}
  download_bars = [
    FloatSlider(
      step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
  ]
  display(*download_bars)

  while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
      bar = download_bars[index + next_shift]
      if not download.is_seed():
        s = download.status()
        bar.description = " ".join(

          [
            download.name(),
            str(s.download_rate / 1000),
            "kB/s",
            state_str[s.state],
          ]
        )
        bar.value = s.progress * 100
      else:
        next_shift -= 1
        ses.remove_torrent(download)
        downloads.remove(download)
        bar.close()  # Not Working
        download_bars.remove(bar)
        print(download.name(), "complete")
    sleep(1)
elif Mode =="Torrent Files DL":
  from google.colab import files
  source = files.upload()
  params = {
    "save_path": Destination,
    "ti": torrent_info(list(source.keys())[0]),
  }
  downloads.append(ses.add_torrent(params))
  from time import sleep
  from IPython.display import display
  from ipywidgets import Layout, FloatSlider
  state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
  ]
  layout = Layout(width="auto")
  style = {"description_width": "initial"}
  download_bars = [
    FloatSlider(
      step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
  ]
  display(*download_bars)

  while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
      bar = download_bars[index + next_shift]
      if not download.is_seed():
        s = download.status()
        bar.description = " ".join(
          [
            download.name(),
            str(s.download_rate / 1000),
            "kB/s",
            state_str[s.state],
          ]
        )
        bar.value = s.progress * 100
      else:
        next_shift -= 1
        ses.remove_torrent(download)
        downloads.remove(download)
        bar.close()  # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
        download_bars.remove(bar)
        print(download.name(), "complete")
    sleep(1)

elif Mode == "Upload file to Anonymous Platforms":
  if not Source:
    end("Source path can't be empty!")
  elif not Path(Source).exists():
    end("Not Found: Given path not exists!")

  API = list(json_data['apis']['anon_file_share'].values())
  site_names = ""

  for num, name in enumerate(list(json_data['apis']['anon_file_share'].keys())):
    site_names += f"{num}.{name}\n"

  while True:
    count = int(input(f"Choose Platform by its number (0 - {len(API) - 1}):\n{site_names}>"))

    if count > len(API) - 1:
      print('Selection out of list.')
    else:
      print(f'Selection -> {count}\nAPI -> {API[count]}\n')
      break

  if not Path(Source).exists():
    end('Not Found: Given source path not exist.')
  elif Path(Source).is_dir():
    links = []
    is_recursive = input("Upload files recursively? 0 = No & 1 = Yes:\n")
    if is_recursive == '1':
      print("Scanning path for all possible directories...")
      for file_path in list(Path(Source).glob('**/*.*')):
      # This check is still important, since Google Colab secretly creates '.ipynb_checkpoints' detected as folder by pathlib
        if Path(file_path).is_file():
          try:
            print(f"Uploading [{PurePath(file_path).name}]")
            api_resp = post_url(API[count], files = {'file': open(file_path,'rb')}).json()
            print(f"{PurePath(file_path).name} -> " + api_resp["data"]["file"]["url"]["full"])
            links.append(f"{PurePath(file_path).name}:\n" + api_resp["data"]["file"]["url"]["full"])
          except Exception as error_code:
            print(f"Unable to upload!\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
            sleep(4)
        else:
          print(f"[Dir] -> {PurePath(file_path).name}")
    else:
      # Path.iterdir() in Python3.8 causes memory leak & slow instead use Path.glob()
      for file_path in list(Path(Source).glob("*.*")):
        if file_path >= 5000:
          break
        if Path(file_path).is_file():
          try:
            print(f"Uploading [{PurePath(file_path).name}]")
            api_resp = post_url(API[count], files = {'file': open(file_path,'rb')}).json()
            print(f"{PurePath(file_path).name} -> " + api_resp["data"]["file"]["url"]["full"])
            links.append(f"{PurePath(file_path).name}:\n" + api_resp["data"]["file"]["url"]["full"])
          except Exception as error_code:
            print(f"Unable to upload!\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
            sleep(4)
        else:
          print(f"[Dir] -> {PurePath(file_path).name}")
                            
    if len(links) > 1:
      print(f"Total Links -> {len(links)}")
      is_export = input("Do you want to export all links in txt file? 0 = No or 1 = Yes:\n")
      if is_export not in ['0','1'] or is_export == '1':
        with open('links.txt','a+') as save_links:
          save_links.writelines('%s\n' % item for item in links)
          save_links.close()
          dl('links.txt')
          print('Request: Initiated download -> links.txt [file]')
      else:
        with open('links.txt','w+') as save_links:
          save_links.ritelines('%s\n' % item for item in links)
          save_links.close()
          print("Saved: links.txt [file] -> Runtime")
    else:
      print(f'Total Links -> {len(links)}')
  elif Path(Source).is_file():
    print(f"Uploading [{PurePath(Source).name}]")
    api_resp = post_url(API[count], files={'file': open(Source, 'rb')}).json()
    print(f"{PurePath(Source).name} -> " + api_resp["data"]["file"]["url"]["full"])

elif Mode =="MEGA Link DL":
  use_account = input('Do you want to use your MEGA Account? to lift limits imposed on anonymous downloader? 0 = No & 1 = Yes:\n')
  if use_account not in ['0','1']:
    end("Input Error: Invalid input parameter! Please enter 0 or 1 as input.")
  elif use_account == '1' and Path('/content/mega.json').exists():
    use_existing = input('[INFO] - Found existing MEGA Account credentials, do you want to use them? 0 = No & 1 = Yes:\n')
    if use_existing not in ['0','1']:
      end("Input Error: Invalid input parameter! Please enter 0 or 1 as input.")
    elif use_existing == '1':
      config_creds()
    elif use_existing == '0':
      save_creds()
  elif use_account == '1':
    save_creds()
  else:
    mega_dl()

elif Mode == "UUP DUMP":
  UUP_Dir = f"{Source}/uup_download_linux.sh"
  if not Path(Source).exists():
    end("Not Found: Given source path not exist.")
  elif not Path(Destination).exists():
    end("Not Found: Given destination path not exist.")
  elif not Path(Destination).is_dir() or Path(Destination).is_mount():
    end("Path Error: Given destionation path is not a valid directory or mount point.")
  elif Path(Source).is_file():
    %cd "$Destination"
    !bash "$Source"
    print('Execution completed!')
  elif Path(Source).is_dir() and Path(UUP_Dir).exists():
    %cd "$Destination"
    !bash "$UUP_Dir"
    print('Execution completed!')     
  else:
    print('Not Found: uup_download_linux.sh [file]')

###**Packing & Extraction:**

In [ ]:
from sys import exit as end
from pathlib import Path, PurePath
from getpass import getpass

# User Input
Action = "Zip" # @param ["Zip", "Zip with Password", "RaR", "RaR with Password", "7z", "7z with Password", "Tar", "Tar.gz", "Tar.bz2", "UnZip", "UnZip with Password", "UnRaR", "UnRaR with Password", "Extract 7z", "Extract 7z with Password", "UnTar", "UnTar.gz", "UnTar.bz2"]
Packed_File_Name = "" #@param {type:"string"}
Source_Path = "" #@param {type:"string"}
Destination_Path = "" #@param {type:"string"}

# Check for Setup & common things
if not Path("/content/setup.sh").exists():
  end('Please download Setup first!')
elif not Action:
  end("Select a supported operation to perform.")
elif not Source_Path:
  end("Source field can't be empty.")
elif not Destination_Path:
  end("Destination field can't be empty")
elif not Path(Source_Path).exists():
  end("Not found: Given source path not exist.")
elif not Path(Destination_Path).exists():
  end("Not found: Given destination path not exist.")
else:
  Source_Path = PurePath(Source_Path)
  Destination_Path = PurePath(Destination_Path)
  if Action in ['Zip', 'Zip with Password', 'UnZip with Password', 'RaR', 'RaR with Password', '7z', '7z with Password', 'Tar', 'Tar.gz', 'Tar.bz2']:
    if not Packed_File_Name:
      Packed_File_Name = PurePath(Source_Path).name
    Pack_Dest = f"{Destination_Path}/{Packed_File_Name}.zip"
    sz_Dest = f"{Destination_Path}/{Packed_File_Name}.7z"
    Tar_Dest = f"{Destination_Path}/{Packed_File_Name}.tar"
    Targz_Dest = f"{Destination_Path}/{Packed_File_Name}.tar.gz"
    Tarbz_Dest = f"{Destination_Path}/{Packed_File_Name}.tar.bz"

if Action == "Zip":
  !zip "$Pack_Dest" "$Source_Path"
elif Action == "Zip with Password":
  password = getpass("Enter password:(Input is hidden)\n")
  !zip -P "$password" "$Pack_Dest" "$Source_Path"
elif Action == "UnZip":
  !unzip "$Source_Path" -d "$Destination_Path"
elif Action == "UnZip with Password":
  password = getpass("Enter password:(Input is hidden)\n")
  !unzip -P "$password" -d "$Destination_Path" "$Source_Path"
elif Action == "RaR":
  !rar a "$Pack_Dest" "$Source_Path"
elif Action == "RaR with Password":
  !rar a -p "$Pack_Dest" "$Source_Path"
elif Action == "UnRaR":
  !unrar x "$Source_Path" "$Destination_Path"
elif Action == "UnRaR with Password":
  !unrar x -Y "$Source_Path" "$Destination_Path"
elif Action == "7z":
  !7z a "$sz_Dest" "$Source_Path"
elif Action == "7z with Password":
  password = getpass("Enter password:(Input is hidden)\n")
  !7z a -p"$password" "$sz_Dest" "$Source_Path"
elif Action == "Extract 7z":
  !7z x "$Source_Path" -o"$Destination_Path"
elif Action == "Extract 7z with Password":
  password = getpass("Enter password:(Input is hidden)\n")
  !7z x -p"$password" -o"$Destination_Path" "$Source_Path"
elif Action == "Tar":
  !tar -cvf "$Tar_Dest" "$Source_Path"
elif Action == "UnTar" :
  !tar -xvf "$Source_Path" -C "$Destination_Path"
elif Action == "Tar.gz":
  !tar -czvf "$Targz_Dest" "$Source_Path"
elif Action == "UnTar.gz":
  !tar -xzvf "$Source_Path" -C "$Destination_Path"
elif Action == "Tar.bz2":
  !tar -cjvf "$Tarbz_Dest" "$Source_Path"
elif Action == "UnTar.bz2":
  !tar -xjvf "$Source_Path" -C "$Destination_Path"
else:
  print("Invalid Operation.")

###**GoFile Integration:**

**Options:**<br>
`RECURSIVE`**: Upload files recursively from folder.**<br>
`RAW`**: Show API Response as raw.**

In [ ]:
# @markdown <br><center><img src="https://gofile.io/dist/img/logo-small-70.png" height="50" alt="Gdrive-logo' "/></center>
# @markdown <center><h3><i><b>GoFile Transfer</b></i></h3></center><br>

from requests import get as get_url, post as post_url, put as put_url
from sys import exit as end
from pathlib import Path, PurePath
from json import load as json_load, dumps as dump_json
from dns.resolver import resolve as detect_dns
from ping3 import ping
from time import sleep
from google.colab import output
from google.colab.files import download as dl
from getpass import getpass


# Retrive Json Data
json_data = get_url(
    "https://gist.githubusercontent.com/TheCaduceus/29789e6972aad136a99b35fe0d9adbde/raw/MCT.json").json()

# Where user mounted Google Drive?
try:
  CMP = PurePath(Custom_Mount_Path)
except NameError:
  CMP = '/content/drive'

# Paths
DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config/"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
  end('Please download Setup first!')
elif not Path(CMP).exists():
  end('Not Found: Google Drive is not connected!')
elif not Path(f'{CMP}/MyDrive/MCT-Config/').exists():
	!rclone mkdir "$DRIVE_CONFIG_DIR"
	print('Created: MCT-Config [folder] -> MyDrive')

METHOD = "Manage Tokens" # @param ["Server Details", "Upload Files", "Manage Tokens"]
SOURCE = "File/Folder Path"  # @param {type:"string"}
DESTINATION = "Parent Folder ID"  # @param {type:"string"}
RECURSIVE = False  # @param {type:"boolean"}
RAW = False  # @param {type:"boolean"}

# Get Fastest Server
try:
  server_id = get_url(json_data["apis"]["GoFile"]["getServer"]).json()["data"]["server"]
except Exception as error_code:
  end(f"An error occured!\nTraceback:\n{error_code}")

# Load configuration
if Path('gofile.json').exists():
  with open('gofile.json', 'r') as conf_file:
    gofile_json = json_load(conf_file)
    conf_file.close()
  print("[INFO] - Decrypting tokens.")
else:
  with open('gofile.json', 'w+') as conf_file:
    gofile_json = {
        "version": "1.0",
        "tokens": {}
    }
    conf_file.write(dump_json(gofile_json))

# Check config file version
if gofile_json["version"] != json_data["apis"]["GoFile"]["config_version"]:
  # This check is for future update and it will be modified to upgrade gofile config file version in case I change its format
  end("GoFile configuration file version mismatch!")

tokens = []

for i in list(gofile_json["tokens"].values()):
  tokens.append(post_url(f'{json_data["apis"]["encrypt_api"]}/decrypt', json = {"string": i}).json()["result"])

profile_string = ""

for num, name in enumerate(list(gofile_json["tokens"].keys()), start=1):
  profile_string += f"{num}.{name}\n"

if METHOD == 'Server Details':
  print(f"Server Details:\nServer ID -> {server_id}\nServer IP -> {detect_dns(json_data['apis']['GoFile']['server_url'].format(server_id=server_id), 'A')[0]}\nServer URL: {json_data['apis']['GoFile']['server_url'].format(server_id=server_id)}\nPing: {round(ping(json_data['apis']['GoFile']['server_url'].format(server_id=server_id)) * 1000, 2)} ms")
elif METHOD == 'Upload Files':
  while True:
    count = int(input(f"Choose Profile by its number (0 - {len(tokens) if len(tokens) >= 1 else 1 - 1}):\n0.Anonymous Upload\n{profile_string}>")) - 1
    if count > len(tokens) - 1:
      print("Selection out of list.")
    else:
      break
  output.clear()
  links = []
  if count == -1:  # For anonymous upload
    if not Path(SOURCE).exists():
      end("Not Found: Given source path not exist!")
    elif Path(SOURCE).is_dir():
      if RECURSIVE:
        print("Scanning path for all possible directories...")
        for file_path in list(Path(SOURCE).glob('**/*.*')):
          # This check is still important, since Google Colab secretly creates '.ipynb_checkpoints' detected as folder by pathlib
          if Path(file_path).is_file():
            try:
              print(f"Uploading [{PurePath(file_path).name}]")
              api_resp = post_url(json_data['apis']['GoFile']['uploadFile'].format(server_id=server_id), files={'file': open(file_path, 'rb')}).json()
              if not RAW:
                print(f"{PurePath(file_path).name} -> " + api_resp['data']['downloadPage'])
              else:
                print(f"{PurePath(file_path).name}:")
                print(api_resp)
              links.append(f"{PurePath(file_path).name}:\n" + api_resp['data']['downloadPage'])
            except Exception as error_code:
              print("Unable to upload!\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
              sleep(4)
          else:
            print(f"[Dir] -> {PurePath(file_path).name}")
      else:
        # Path.iterdir() in Python3.8 causes memory leak & slow instead use Path.glob()
        for file_path in list(Path(SOURCE).glob("*.*")):
          if Path(file_path).is_file():
            try:
              print(f"Uploading [{PurePath(file_path).name}]")
              api_resp = post_url(json_data['apis']['GoFile']['uploadFile'].format(server_id=server_id), files={'file': open(file_path, 'rb')}).json()
              if not RAW:
                print(f"{PurePath(file_path).name} -> " + api_resp['data']['downloadPage'])
              else:
                print(f"{PurePath(file_path).name}:")
                print(api_resp)
              links.append(f"{PurePath(file_path).name}:\n" + api_resp['data']['downloadPage'])
            except Exception as error_code:
              print(f"Unable to upload!\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
              sleep(4)
          else:
            print(f"[Dir] -> {PurePath(file_path).name}")
    elif Path(SOURCE).is_file():
      try:
        print(f"Uploading [{PurePath(SOURCE).name}]")
        api_resp = post_url(json_data['apis']['GoFile']['uploadFile'].format(server_id=server_id), files={'file': open(SOURCE, 'rb')}).json()
        if not RAW:
          print(f"{PurePath(SOURCE).name} -> " + api_resp['data']['downloadPage'])
        else:
          print(f"{PurePath(SOURCE).name}:")
          print(api_resp)
      except Exception as error_code:
        print(f"Unable to upload!\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
  else:  # For upload with specific token & folder
    if not Path(SOURCE).exists():
      end("Not Found: Given source path not exist.")
    elif not DESTINATION:
      end("Not provided parent folder id to upload in.")
    elif Path(SOURCE).is_dir():
      if RECURSIVE:
        print("Scanning path for all possible directories...")
        for file_path in list(Path(SOURCE).glob('**/*.*')):
          # This check is still important, since Google Colab secretly creates '.ipynb_checkpoints' detected as folder by pathlib
          if Path(file_path).is_file():
            try:
              print(f"Uploading [{PurePath(file_path).name}]")
              api_resp = post_url(json_data['apis']['GoFile']['uploadFile'].format(server_id=server_id), files={'file': open(file_path, 'rb')}, data={'token': tokens[count], 'folderId': DESTINATION}).json()
              if not RAW:
                print(f"{PurePath(file_path).name} -> " + api_resp['data']['downloadPage'])
              else:
                print(f"{PurePath(file_path).name}:")
                print(api_resp)
              links.append(f"{PurePath(file_path).name}:\n" + api_resp['data']['downloadPage'])
            except Exception as error_code:
              print(f"Unable to upload!\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
              sleep(4)
          else:
            print(f"[Dir] -> {PurePath(file_path).name}")
      else:
        # Path.iterdir() in Python3.8 causes memory leak & slow instead use Path.glob()
        for file_path in list(Path(SOURCE).glob("*.*")):
          if Path(file_path).is_file():
            try:
              print(f"Uploading [{PurePath(file_path).name}]")
              api_resp = post_url(json_data['apis']['GoFile']['uploadFile'].format(server_id=server_id), files={'file': open(file_path, 'rb')}, data={'token': tokens[count], 'folderId': DESTINATION}).json()
              if not RAW:
                print(f"{PurePath(file_path).name} -> " + api_resp['data']['downloadPage'])
              else:
                print(f"{PurePath(file_path).name}:")
                print(api_resp)
              links.append(f"{PurePath(file_path).name}:\n" + api_resp['data']['downloadPage'])
            except Exception as error_code:
              print(f"Unable to upload!\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")
              sleep(4)
          else:
            print(f"[Dir] -> {PurePath(file_path).name}")
    elif Path(SOURCE).is_file():
      print(f"Uploading [{PurePath(SOURCE).name}]")
      try:
        api_resp = post_url(json_data['apis']['GoFile']['uploadFile'].format(server_id=server_id), files={'file': open(SOURCE, 'rb')}, data={'token': tokens[count], 'folderId': DESTINATION}).json()
        if not RAW:
          print(f"{PurePath(SOURCE).name} -> " + api_resp['data']['downloadPage'])
        else:
          print(f"{PurePath(SOURCE).name}:")
          print(api_resp)
      except Exception as error_code:
        print(f"Unable to upload!\nAPI Response:\n{api_resp}\nTraceback:\n{error_code}")

  if len(links) > 1:
    print(f"Total Links -> {len(links)}")
    is_export = input("Do you want to export all links in txt file? 0 = No or 1 = Yes:\n")
    if is_export not in ['0', '1'] or is_export == '1':
      with open('gofile-links.txt', 'a+') as save_links:
        save_links.writelines('%s\n' % item for item in links)
        save_links.close()
      dl('gofile-links.txt')
      print('Request: Initiated download -> gofile-links.txt [file]')
    else:
      with open('gofile-links.txt', 'w+') as save_links:
        save_links.writelines('%s\n' % item for item in links)
        save_links.close()
      print("Saved: gofile-links.txt [file] -> Runtime")

elif METHOD == 'Manage Tokens':
  print(f"Availabe tokens: {len(tokens)}\n{profile_string}")
  while True:
    action = input("a/r/e>")
    if action in ['a', 'r', 'add', 'remove', 'rm', '+', '-','e']:
      break

  if action in ['a', 'add', '+']:
    token_name = input("Give a name to your token:\n>")
    token_value = getpass("Enter your token value: (input is hidden)\n")
    print("[INFO] - Encryption token.")
    token_value = post_url(f'{json_data["apis"]["encrypt_api"]}/encrypt', json = {"string": token_value}).json()["result"]
    gofile_json['tokens'][token_name] = token_value
    with open('gofile.json', 'w+') as conf_file:
      conf_file.write(dump_json(gofile_json))
      conf_file.close()
      print(f"Saved [{token_name}].")
    !rclone copy "gofile.json" "$DRIVE_CONFIG_DIR"
  elif action in ['r', 'remove', 'rm', '-']:
    token_name = input("Enter name of token you want to delete:\n>")
    try:
      del gofile_json['tokens'][token_name]
    except KeyError:
      end("Given token name not exist!")
    with open('gofile.json', 'w+') as conf_file:
      conf_file.write(dump_json(gofile_json))
      conf_file.close()
      print(f"Deleted [{token_name}].")
    !rclone copy "gofile.json" "$DRIVE_CONFIG_DIR"
  else:
    pass


## **Advanced Task: Run Commands**

**You can run any supported command below after 
downloading its setup from above!<br>List of Commands:**
```
# Putting '!' before any command is mandatory!
---
!rclone config - Enter an interactive configuration session.
!rclone copy - Copy files from source to dest, skipping already copied.
!rclone sync - Make source and dest identical, modifying destination only.
!rclone move - Move files from source to dest.
!rclone delete - Remove the contents of path.
!rclone purge - Remove the path and all of its contents.
!rclone mkdir - Make the path if it doesn't already exist.
!rclone rmdir - Remove the path.
!rclone rmdirs - Remove any empty directories under the path.
!rclone check - Check if the files in the source and destination match.
!rclone ls - List all the objects in the path with size and path.
!rclone lsd - List all directories/containers/buckets in the path.
!rclone lsl - List all the objects in the path with size, modification time and path.
!rclone md5sum - Produce an md5sum file for all the objects in the path.
!rclone sha1sum - Produce a sha1sum file for all the objects in the path.
!rclone size - Return the total size and number of objects in remote:path.
!rclone version - Show the version number.
!rclone cleanup - Clean up the remote if possible.
!rclone dedupe - Interactively find duplicate files and delete/rename them.
!rclone authorize - Remote authorization.
!rclone cat - Concatenate any files and send them to stdout.
!rclone copyto - Copy files from source to dest, skipping already copied.
!rclone genautocomplete - Output shell completion scripts for rclone.
!rclone gendocs - Output markdown docs for rclone to the directory supplied.
!rclone listremotes - List all the remotes in the config file.
!rclone mount - Mount the remote as a mountpoint.
!rclone moveto - Move file or directory from source to dest.
!rclone obscure - Obscure password for use in the rclone.conf
!rclone cryptcheck - Check the integrity of an encrypted remote.
!rclone about - Get quota information from the remote.
---
!yt-dlp [URL-1 URL-2 URL-3] [OPTIONS]
---
!aria2c <torrent/magnet/url>
```

**Write your Code Below:**

**Code 1**

In [ ]:
  #<--Write Code before #

**Code 2**

In [ ]:
  #<--Write Code before #

**Powered By:<br>**
```
---
Base
---
NoteBook: v5.0.2
Python: v3.8
Rclone: v1.61.1
Installer: v5.0.1 (R-1620)
---
Zip Handlers
---
7-Zip: v22.07.15
WinRAR: v6.20
---
Add-Ons
---
yt-dlp: v2023.01.16
aria2: v1.36.0
---
Torrent Manager
---
LibTorrent: v2.0.8

```